In [314]:
import csv
import os.path
import configparser as ConfigParser
import pandas as pd
import numpy as np
import requests
import csv
import urllib.parse
import re
from urllib import parse

In [315]:
#UCUM_SERVICE ='http://dataportals.pangaea.de/test/ucum/?ucum='
UCUM_SERVICE_VALIDATE ='http://seprojects.marum.de:3838/pucum/v1/api/validate/'
UCUM_SERVICE_QUANTITY ='http://seprojects.marum.de:3838/pucum/v1/api/quantity/'

# Read Files

In [316]:
# These should be the full file paths or at least relative to the cwd
file_cfg = ConfigParser.ConfigParser()
file_cfg.read('C:\\Users\\asd\\python-workspace\\zucchini\\config\\zucchini.ini')

['C:\\Users\\asd\\python-workspace\\zucchini\\config\\zucchini.ini']

In [317]:
main_dir = os.path.dirname(os.getcwd())
param_file = os.path.join(main_dir, file_cfg.get("Input", "input_file"))
param_file, main_dir

('C:\\Users\\asd\\python-workspace\\zucchini\\input\\PANGAEAParameterComplete.tab',
 'C:\\Users\\asd\\python-workspace\\zucchini')

In [318]:
cols_params = ['Parameter', 'Abbreviation','Unit','ID parameter']
df = pd.read_csv(param_file, header=0, usecols=cols_params, delimiter="\t", encoding='utf-8')

In [319]:
df.head()

,Parameter,Abbreviation,Unit,ID parameter
0,-,NaN,NaN,16431
1,-,NaN,NaN,16432
2,10:2 fluorotelomer alcohol,10:2 FTOH,pg/m**3,53014
3,10:2 fluorotelomer alcohol,10-2FTOH,pg/l,159280
4,"10-Methyldodecanoic acid, particulate",10-methyllauric acid part,nmol/l,30245


In [320]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 158025 entries, 0 to 158024
Data columns (total 4 columns):
Parameter       158025 non-null object
Abbreviation    158022 non-null object
Unit            122269 non-null object
ID parameter    158025 non-null int64
dtypes: int64(1), object(3)
memory usage: 4.8+ MB


In [321]:
#remove lines with no parameter
df = df[df.Parameter != '-']

In [322]:
df.head(2)

,Parameter,Abbreviation,Unit,ID parameter
2,10:2 fluorotelomer alcohol,10:2 FTOH,pg/m**3,53014
3,10:2 fluorotelomer alcohol,10-2FTOH,pg/l,159280


In [323]:
(df.Unit.str.contains('nan', flags=re.IGNORECASE, na=False,regex=True)).unique()

array([False], dtype=bool)

In [324]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 158023 entries, 2 to 158024
Data columns (total 4 columns):
Parameter       158023 non-null object
Abbreviation    158022 non-null object
Unit            122269 non-null object
ID parameter    158023 non-null int64
dtypes: int64(1), object(3)
memory usage: 6.0+ MB


In [325]:
#List unique values each cols
df.nunique(dropna=True)#Excludes NA values

Parameter       127375
Abbreviation    119256
Unit              1337
ID parameter    158023
dtype: int64

In [326]:
df.Unit.unique()[:30]

array(['pg/m**3', 'pg/l', 'nmol/l', 'µg/g', 'per mil PDB', '%', 'pptv',
       'ng/g', 'pmol/l', 'ng/l', 'pg/g', nan, 'ppbv', 'pg/ml', 'nmol/kg',
       'ng/NT', 'per mil', 'µg C/l/day', 'mg C/kg/day', 'nmol/l/day',
       'pmol/#', 'dpm', '±', 'fg/day', 'ng/l/day', 'mg/m**2/day',
       'pmol/min/mg', '#/cm**3', 'µg/kg', 'µg/l'], dtype=object)

In [327]:
df.Unit.isnull().values.any()

True

In [328]:
list_of_uom = df.Unit.dropna().unique()

In [329]:
len(list_of_uom)

1337

In [330]:
df["Ucum"] = np.nan
df["QuantityKind"] = np.nan
df["Dimension"] = np.nan
df["CanonicalUnits"] = np.nan
df.head(7)

,Parameter,Abbreviation,Unit,ID parameter,Ucum,QuantityKind,Dimension,CanonicalUnits
2,10:2 fluorotelomer alcohol,10:2 FTOH,pg/m**3,53014,NaN,NaN,NaN,NaN
3,10:2 fluorotelomer alcohol,10-2FTOH,pg/l,159280,NaN,NaN,NaN,NaN
4,"10-Methyldodecanoic acid, particulate",10-methyllauric acid part,nmol/l,30245,NaN,NaN,NaN,NaN
5,10-methyl-Hexadecanoic acid,10Me-C16:0,µg/g,46516,NaN,NaN,NaN,NaN
6,"10-methyl-Hexadecanoic acid, d13C",10Me-C16:0 d13C,per mil PDB,46530,NaN,NaN,NaN,NaN
7,10-methyl-Hexadecanoic acid of total fatty acids,10me16:0,%,97842,NaN,NaN,NaN,NaN
8,10-methyl-Octadecanoic acid of total fatty acids,10me18:0,%,97853,NaN,NaN,NaN,NaN


# Get Quantity Kind

In [331]:
specialChars = {
'#':'1',
'µ':'u',
}

In [332]:
def pattern_match(m):
    m = m.group(0).replace('*', '')
    return m
pattern  = re.compile(r'[A-z]+[*]+\d') #pg/m**3 --> pg/m3; 10**6 ru/l --> 10**6 ru/l

In [333]:
#Test pattern
pattern.sub(pattern_match, 'pg/m**3'),pattern.sub(pattern_match, '10**3'),pattern.sub(pattern_match, '1m**31')

('pg/m3', '10**3', '1m31')

In [334]:
np.where(df['Unit'] == 'pg/m**3')

(array([     0,     42,    303,    307,    606,    608,    610,    627,
           629,   5715,   6238,  27448,  30011,  31956,  44544,  45429,
         50616,  58913,  68765,  69729,  76201,  79377,  89985,  89998,
         90020,  90033,  90052,  90065,  90100,  90114,  90134,  90148,
         90167,  90179,  90191,  92285,  93106,  93107,  93108,  93110,
         94449,  94450,  94451,  94453,  94455,  94457,  99513,  99515,
         99520, 100051, 102485, 102492, 102498, 103961, 104169, 104181,
        104188, 104197, 104202, 104217, 110116, 110117, 110121, 110125,
        110127, 110129, 110131, 110134, 110139, 110142, 110145, 110147,
        110151, 110153, 110157, 110160, 110330, 142996, 142998, 143069,
        143080, 143085, 143086, 149100, 150397, 150743, 151259, 151520,
        151759, 151893, 151896, 151899, 151900, 151903], dtype=int64),)

In [335]:
#df['QuantityKind'] = df['QuantityKind'].astype(str)
df.dtypes

Parameter          object
Abbreviation       object
Unit               object
ID parameter        int64
Ucum              float64
QuantityKind      float64
Dimension         float64
CanonicalUnits    float64
dtype: object

In [336]:
myList = ['a','b']
myString = ",".join(myList )
myString

'a,b'

In [337]:
from urllib import parse
new_url = 'µg/g'
urllib.parse.quote(new_url) 

'%C2%B5g/g'

In [338]:
quantity_not_found=[]
canon_not_found = []
quantity_found =[]
for u in list_of_uom:
    uom = u
    for word, initial in specialChars.items():
        uom = uom.replace(word, initial)
    if '*' in uom:
        uom = pattern.sub(pattern_match, uom)
    q = UCUM_SERVICE_QUANTITY+ urllib.parse.quote(uom) 
    resp = requests.get(q)
    data = resp.json()
    ucumcode = data['ucum']
    if (resp.status_code == requests.codes.ok) or (resp.status_code == requests.codes.BAD_REQUEST):
        canon = data['canonicalunit']
        if canon:
            dimension = data['dimension']
            quantity = data['quantities']
            if isinstance(quantity, list) and len(quantity) > 0:
                quantity_str = ', '.join(quantity)
                quantity_found.append(u)
            df.loc[df.Unit == u, 'QuantityKind'] = quantity_str
            df.loc[df.Unit == u, 'Dimension'] = dimension
            df.loc[df.Unit == u, 'CanonicalUnits'] = canon
            #print(u, " ",uom," ",ucumcode," ",quantity_str, " ",canon," ",dimension )
        else:
            print('******', uom)
            canon_not_found.append(u)
            
    else:
        quantity_not_found.append(u)
    df.loc[df.Unit == u, 'Ucum'] = ucumcode
            

In [339]:
canon_not_found, len(quantity_found)

([], 671)

In [340]:
df[df['CanonicalUnits'].isnull()].shape

(103299, 8)

In [341]:
df[df['QuantityKind'].notnull()].shape

(54724, 8)

In [342]:
len(set(quantity_not_found)), len(quantity_not_found), list(quantity_not_found)[:10]

(503,
 503,
 ['µg/g',
  'per mil PDB',
  '%',
  'pptv',
  'ng/g',
  'pg/g',
  'ppbv',
  'ng/NT',
  'per mil',
  'µg C/l/day'])

In [343]:
df.head()

,Parameter,Abbreviation,Unit,ID parameter,Ucum,QuantityKind,Dimension,CanonicalUnits
2,10:2 fluorotelomer alcohol,10:2 FTOH,pg/m**3,53014,pg/m3,Density,M.L-3,g.m-3
3,10:2 fluorotelomer alcohol,10-2FTOH,pg/l,159280,pg/l,Density,M.L-3,g.m-3
4,"10-Methyldodecanoic acid, particulate",10-methyllauric acid part,nmol/l,30245,nmol/l,"Molar Concentration, Amount of Substance Per U...",L-3.N,m-3.mol
5,10-methyl-Hexadecanoic acid,10Me-C16:0,µg/g,46516,ug/g,NaN,NaN,NaN
6,"10-methyl-Hexadecanoic acid, d13C",10Me-C16:0 d13C,per mil PDB,46530,[per_mil]{PDB},NaN,NaN,NaN


In [344]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 158023 entries, 2 to 158024
Data columns (total 8 columns):
Parameter         158023 non-null object
Abbreviation      158022 non-null object
Unit              122269 non-null object
ID parameter      158023 non-null int64
Ucum              115981 non-null object
QuantityKind      54724 non-null object
Dimension         54724 non-null object
CanonicalUnits    54724 non-null object
dtypes: int64(1), object(7)
memory usage: 10.9+ MB


In [345]:
df_quantity_null = df.loc[df['QuantityKind'].isnull(), ['ID parameter','Parameter','Unit', 'Ucum' ]]

In [346]:
df_quantity_null.head(10)

,ID parameter,Parameter,Unit,Ucum
5,46516,10-methyl-Hexadecanoic acid,µg/g,ug/g
6,46530,"10-methyl-Hexadecanoic acid, d13C",per mil PDB,[per_mil]{PDB}
7,97842,10-methyl-Hexadecanoic acid of total fatty acids,%,%
8,97853,10-methyl-Octadecanoic acid of total fatty acids,%,%
9,97840,10-methyl-pentadecanoic acid of total fatty acids,%,%
10,97837,10-methyl-tetradecanoic acid of total fatty acids,%,%
11,46518,"11,12-methylene-Hexadecanoic acid",µg/g,ug/g
12,100166,"11,12-methylene-Hexadecanoic acid",µg/g,ug/g
13,46532,"11,12-methylene-Hexadecanoic acid, d13C",per mil PDB,[per_mil]{PDB}
14,146077,"1,1,1,2-Tetrafluoroethane",pptv,[pptv]


In [347]:
df_quantity_null.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 103299 entries, 5 to 158024
Data columns (total 4 columns):
ID parameter    103299 non-null int64
Parameter       103299 non-null object
Unit            67545 non-null object
Ucum            61257 non-null object
dtypes: int64(1), object(3)
memory usage: 3.9+ MB


In [348]:
import time
timestr = time.strftime("%Y%m%d%H%M%S")
outputFile = os.path.join(main_dir, "Output"+"\params_qnull_"+timestr+".csv")
outputFile

'C:\\Users\\asd\\python-workspace\\zucchini\\Output\\params_qnull_20180712162553.csv'

In [349]:
df_quantity_null.to_csv(outputFile, sep='\t', encoding='utf-8')